# Optimisation Assignment 1

## Solution for Assignment 1: Unit Commitment Problem Using Linear Programming

By writing my/our name/s below and submitting this file, I/we declare that this file is my/our own work, and that I/we have not seen any work on this assignment by another student/group.

#### Student name(s): Kalyani Prashant Kawale, Harshitha Bengaluru Raghuram
#### Student ID(s): 21237189, 21235396

## Formulation:

### Unit Commitment (Electricity Generation Scheduling) Formulation:

Given the data in generators_info.csv, demand.csv and solar_curve.csv, the formulation is,

#### Objective Function:
Minimise the total cost of electricity produced by all the generators for each hour. 

**To minimise Σ a<sub>i</sub> * x<sub>i,j</sub> ∀ i and j** where,
 
i = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9} where indexes 0 to 9 corresponds to generators [A, B, C, D, E, F, G, H, I, J]

j = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23} 

#### Decision Variables:
Let x<sub>ij</sub> be the supply in MW by generator i at hour j,

i = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

j = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23} 

#### Constraints:
**1. Capacity Constraint:**

Supply of each generator i should be between lower and upper bound.

**lb<sub>i</sub> ≤ x<sub>ij</sub> ≤ ub<sub>i</sub> ∀ i**, 

lb<sub>i</sub> = minimum electricity (in MW) that can be produced by generator i

ub<sub>i</sub> = maximum electricity (in MW) that can be produced by generator i

**2. Solar Constraint:**

Capacity of the electricity produced from the solar generators depends on the time of the day.

**x<sub>ij</sub> ≤ δ<sub>j</sub> * ub<sub>i</sub> ∀ i and j**,

i ∈ {8,  9} corresponding to solar generators [I, J]

δ<sub>j</sub> = percentage of electricity that can be achieved at hour j

ub<sub>i</sub> = maximum MW that can be generated by generator i

**3. Solid Constraint:**

The output from the solid generator remains constant for each hour.

**x<sub>i0</sub> == x<sub>ij</sub> ∀ i and j**,

i ∈ {4, 5, 6} corresponding to solid generators [E, F, G]

**4. Demand Constraint:**

Supply from all generators at a given hour should be equal to the demand for the electricity at that hour.

**Σ x<sub>ij</sub> = d<sub>j</sub> ∀ i and j** where,
 
d<sub>i</sub> = electricity demand at j<sup>th</sup> hour.

##### Note: Ignoring CO<sub>2</sub> emissions, since it is not affecting the cost/demand


## Code:

In [1]:
# Importing pywraplp module of or-tools to create a solver to perform linear programming
from ortools.linear_solver import pywraplp
# Importing pandas to read given csv files
import pandas as pd

def create_decision_variables(solver, generators_info, generators, hours):
    """
        Method to create decision variables for Unit Commitment Problem
    """    
    # Initializing Lower Bound
    lb = 0.0
    # Initializing Upper Bound
    ub = 0.0
    
    # Initializing decision variables for each generator at every hour of the day
    # Constraint Number 1. Setting the lower and upper limit on production of electricity per hour for each generator
    x = {} # x: dictionary of decision variables
    for i in generators:
        lb = float(generators_info.iloc[i]['lower_bound (MW)'])
        ub = float(generators_info.iloc[i]['upper_bound (MW)'])
        for j in hours:
            x[i,j] = solver.NumVar(lb, ub, "x[%i,%i]" % (i, j))
    return x


def add_constraints(solver, x, hours, generators, generators_info):
    """
        Method to add all constraints on the electricity generation to the solver
    """    
    ## Constraint 1. lower and upper limit on production of electricity per hour for each generator 
    # was set while creation of decision variables 
    
    ## Constraint 2. Solar generator supply constraint for each hour j
    # x[solar_generator(i), j] <= solar_generator_supply(j) * generator's max supply (upper bound)
    # Reading solar generator supply percent relative to maximum for each hour j
    solar_generator_supply = pd.read_csv('solar_curve.csv', header=None)
    # Initializing solar_generators using generator indexes corresponding to solar generator I and J
    solar_generators = [8, 9]
    for i in solar_generators:
            for j in hours:
                max_supply_per_hour = solar_generator_supply.iloc[j][0] * generators_info.iloc[i]['upper_bound (MW)']
                solver.Add(x[i, j] <= max_supply_per_hour, 
                           name="limit on supply of solar generator "
                           + generators_info.iloc[i]['name'] + " at hour " + str(j))
    
    
    ## Constraint 3: Solid fuel generators have same production every hour
    # x[solid_generator(i), 0] = x[solid_generator(i), j] for all j
    # Initializing solid_generators using generator indexes corresponding to solar generator E, F and G
    solid_generators = [4, 5, 6]
    for i in solid_generators:
        for j in hours:
            solver.Add(x[i, 0] == x[i, j], 
                       name="equal amount of supply for solid generator " 
                       + generators_info.iloc[i]['name'])

    
    ## Constraint 4. Electricity demand at each hour
    # reading demand.csv file using pandas to get the demand at each hour j
    demand = pd.read_csv('demand.csv', header=None)
    # Sum of MW generated by each generator i == demand at hour j,
    for j in hours:
        solver.Add(x[0, j] + x[1, j] + x[2, j] + x[3, j] + x[4, j] + x[5, j] + x[6, j] + x[7, j] + x[8, j] + x[9,j] 
                   == demand.iloc[j][0], name="limit on electricity generation for hour " + str(j))

def set_objective_and_solve(solver, x, generators, hours, generators_info):
    """ 
        Method to set the objective function to minimize the cost of electricity generation at each hour
        and print the solution
    """ 
    objective = solver.Objective()
    # Setting the coefficient a(i) (Cost per MW produced) for each generator    
    for i in generators:
        cost_per_mw = generators_info.iloc[i]['cost (EUR/MW)']
        for j in hours:            
            objective.SetCoefficient(x[i, j], cost_per_mw)
    # Minimize the objective function
    objective.SetMinimization()
    
    ## Solve the Unit Commitment LP
    return solver.Solve(), objective
    
        
def sensitivity_analysis(solver):
    """
        Method to get sensitivity information from the solved LP
    """    
    binding = []
    # looping through all constraints and identifying the binding constraints    
    for constraint, activity in zip(solver.constraints(), solver.ComputeConstraintActivities()):      
        eps = 0.0000001
        # Checking if the shadow price is greater than eps (shadow price != 0 for binding constraints)        
        # Saving the name, activity and shadow price for each constraint
        if abs(constraint.DualValue()) > eps:            
            binding.append({"name":constraint.name(), "activity": activity, "shadow_price": constraint.DualValue()})            
        
    # Saving the binding constraints in a csv file using pandas
    binding_df = pd.DataFrame.from_dict(binding)
    binding_df.to_csv('binding_constraints.csv', index=False)

    
def UnitCommitmentLP():
    # Instantiating LP solver using the Google's Linear Programming solver for Unit Commitment Problem
    solver = pywraplp.Solver('UnitCommitment', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    
    # Reading all the generator information from the given generator_info.csv file using pandas
    generators_info = pd.read_csv('generator_info.csv')
    
    # Creating a list of 10 generators where the generators A through J are represented by numbers 0 to 9
    generators = list(range(10))
    print("Generators:", generators)
    
    # Creating a list of all hours in a day, using range, starting from hour 0 and ending on hour 23  
    hours = list(range(24))
    print("Hours:", hours)
    
    # Getting the decision variables for optimising electricity generation by all generators in a day
    x = create_decision_variables(solver, generators_info, generators, hours)
    print("Number of decision variables:", solver.NumVariables())

    # Adding the constraints on electricity generation
    add_constraints(solver, x, hours, generators, generators_info)
    print("Number of constraints:", solver.NumConstraints())
    
    # Setting objective function and solving the LP
    result, objective = set_objective_and_solve(solver, x, generators, hours, generators_info)
    
    solution = []
    # Checking if the solution is optimal and printing the values for decision variables and the objective function
    if result == solver.OPTIMAL:
        print("\nSOLUTION FOR LP:")
        # Printing and saving values of decision variables into solution list
        print("Values of 240 decision variables:")
        for j in hours:
            row = {}
            for i in generators:
                row[generators_info.iloc[i]['name']] = x[i, j].solution_value()
                print(f"{x[i, j]} = {x[i, j].solution_value()} MW;\t Coefficient {objective.GetCoefficient(x[i, j])} EUR/MW;\t Reduced cost {x[i, j].reduced_cost()} EUR")
            solution.append(row)
        print(f"\nOptimal objective value:", objective.Value(), "EUR")
    else:
        print("Problem is not feasible")
        
    # Saving decision variables to a csv file
    decision_variable_df = pd.DataFrame(solution)    
    decision_variable_df.to_csv('decision_variables.csv', index=False)
            
    # Performing Sensitivity Analysis
    sensitivity_analysis(solver)
    
if __name__ == "__main__":
    UnitCommitmentLP()

Generators: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Hours: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
Number of decision variables: 240
Number of constraints: 144

SOLUTION FOR LP:
Values of 240 decision variables:
x[0,0] = 68.99999999999997 MW;	 Coefficient 1.4 EUR/MW;	 Reduced cost 4.440892098500626e-16 EUR
x[1,0] = 80.0 MW;	 Coefficient 1.4 EUR/MW;	 Reduced cost 4.440892098500626e-16 EUR
x[2,0] = 60.0 MW;	 Coefficient 1.4 EUR/MW;	 Reduced cost 4.440892098500626e-16 EUR
x[3,0] = 10.0 MW;	 Coefficient 1.4 EUR/MW;	 Reduced cost 4.440892098500626e-16 EUR
x[4,0] = 442.0 MW;	 Coefficient 4.4 EUR/MW;	 Reduced cost 6.128431095930864e-14 EUR
x[5,0] = 600.0 MW;	 Coefficient 4.4 EUR/MW;	 Reduced cost 6.128431095930864e-14 EUR
x[6,0] = 100.0 MW;	 Coefficient 4.4 EUR/MW;	 Reduced cost 6.128431095930864e-14 EUR
x[7,0] = 100.0 MW;	 Coefficient 9.1 EUR/MW;	 Reduced cost 7.7 EUR
x[8,0] = 0.0 MW;	 Coefficient 6.4 EUR/MW;	 Reduced cost 5.000000000000001 EUR
x[9,0] = 

## Solution:

From the output of above code we get the following results:

1. Objective function value i.e. **minimum cost required to meet the electricity supply demand for a day** is **151504.5 EUROS**

#### Values of decision variables are as follows:

In [ ]:
# Visualizing the decision variables as a heatmap

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Reading decision variables from the saved csv file
decision_variables = pd.read_csv("decision_variables.csv")
print("Decision Variable Values:\n", decision_variables)

# Setting the heatmap figure
plt.figure(figsize=(12, 12))
plt.title("Decision Variables calcuated by LP for Unit Commitment Problem; Minimised cost = 151504.5 Euros")
p=sns.heatmap(decision_variables, annot=True, cmap ='YlGnBu', linewidths=0.1)


2. From the sensitivity analysis data provided by or-tools we get the following information,
    - Out of 144 constraints, 99 are binding.
    - All the constraints for demand at any given hour are binding and changing any of the demand by 1 unit or 1 MW        would reduce the total cost by the subsequent shadow price.
    - Similarly, all constraints for solid fuel generators, and the constraints on solar generator I and solar generator J for hours 19, 20 and 21 i.e. x[I, 19] <= 0.5 * 70, x[I, 20] <= 0.5 * 70, x[I, 21] <= 0.5 * 70, x[I, 19] <= 0.5 * 20, x[I, 20] <= 0.5 * 20 and x[I, 21] <= 0.5 * 20 are also preventing the cost from minimising.
    
#### Binding Constraints for Unit Commitment Problem with their shadow prices:

In [66]:
# Reading constraints from binding constraints csv file and printing the dataframe
binding = pd.read_csv("binding_constraints.csv")
pd.set_option("display.max_rows", len(binding))
print(binding)

                                               name  activity  shadow_price
0   limit on supply of solar generator I at hour 19       0.0          -2.7
1   limit on supply of solar generator I at hour 20       0.0          -2.7
2   limit on supply of solar generator I at hour 21       0.0          -2.7
3   limit on supply of solar generator J at hour 19       0.0          -2.7
4   limit on supply of solar generator J at hour 20       0.0          -2.7
5   limit on supply of solar generator J at hour 21       0.0          -2.7
6      equal amount of supply for solid generator E       0.0          -3.0
7      equal amount of supply for solid generator E       0.0          -3.0
8      equal amount of supply for solid generator E       0.0          -3.0
9      equal amount of supply for solid generator E       0.0          -3.0
10     equal amount of supply for solid generator E       0.0          -3.0
11     equal amount of supply for solid generator E       0.0          -3.0
12     equal

## Acknowledgements
1. https://developers.google.com/optimization/lp/stigler_diet on 18 October 2021
2. https://developers.google.com/optimization/lp/lp_example on 18 October 2021
3. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_dict.html on 19 October 2021
4. https://pandas.pydata.org/docs/user_guide/options.html on 19 October 2021
5. https://seaborn.pydata.org/generated/seaborn.heatmap.html on 20 October 2021